<font size="7" face="monospace">Graduation Topic</font>


<br>
<font size="5" face="monospace">
    資料來源：http://mops.twse.com.tw/mops/web/t163sb04
</font>

<br>
<font size="5" face="monospace">
    篩選標準：eps, 營收, 毛利淨額
</font>


<br>
<font size="5" face="monospace">
    飆股定義：超過一倍
</font>

<br>
<font size="5" face="monospace">
    流程：<br><br>
            1. 設置年份 -> multicore_growth -> growth -> multicore_season -> season <br><br>
            
            2. season -> elec -> category ->season<br><br>
            
            3. season -> multicore_season -> growth <br><br>
            
            4. growth -> compare_year -> multicore_growth<br><br>
            
            5. multicore_growth -> eps or income or profit -> compare_season -> eps or income or profit<br><br>
            
            6. eps or income or profit -> intersection -> excel<br><br>
</font>

<br>
<font size="5" face="monospace">
    # def multicore_growth：啟動多進程，執行growth
    
    # def multicore_season：啟動多進程，執行season
    
    # def elec：選出電子類股
    
    # def category：篩選要的類別，eg. eps, 營收, 毛利淨額
    
    # def compare_year：將兩個年度的公司名單刪減成一樣的表格，且順序也一樣
    
    # def compare_season：將三個季度的公司名單刪減成一樣的表格，且順序也一樣
    
    # def season：找出指定年份三個季度的資料
    
    # def growth：計算eps, profit, income之成長率
    
    # def eps, profit, income：篩選出連三季成長之公司
    
    # def intersection：取eps, profit, income之交集公司
    
    # def excel：輸出成excel
</font>
<br>

In [1]:
import requests
import pandas as pd
import multiprocessing as mp
from functools import partial

url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
head = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36'}
payload = {
    'encodeURIComponent':'1',
    'step':'1',
    'firstin':'1',
    'off':'1',
    'TYPEK':'sii',
    'year':'',
    'season':''
}

In [2]:
def inputs(raw):

    y_start = int(raw[0])
    y_end = int(raw[2])
    s_start = int(raw[1])
    s_end = int(raw[3])
    
    outputs = []
    
    #跨年度
    if y_end - y_start == 1:
        
        #start
        s = 4 - s_start + 1
        for i in range(s):
            outputs.append([y_start, s_start])
            s_start += 1
            
        #end
        s = 1
        for i in range(s_end):
            outputs.append([y_end, s])
            s += 1      
        
        return outputs
    
    #同年度
    elif y_end == y_start:
        s = s_end - s_start + 1
        for i in range(s):
            outputs.append([y_start, s_start])
            s_start += 1
            
        return outputs

In [3]:
def elec(raw):
    
    #電子類股：公司代號
    #資料來源:GoodInfo  https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=全部&INDUSTRY_CAT=電子零組件業&SHEET=交易狀況&SHEET2=日&RPT_TIME=最新資料
    elec_companies = [
        #電子零組件
            1471,	    1582,	    2059,	    2308,	    2313,	    2316,	    2327,	    2328,	    2355,	    2367,	    2368,	    2375,	    2383,	    2385,	    2392,	    2402,	    2413,	    2415,	    2420,	    2421,	    2428,	    2429,	    2431,	    2440,	    2443,	    2456,	    2457,	    2460,	    2462,	    2467,	    2472,	    2476,	    2478,	    2483,	    2484,	    2492,	    2493,	    3003,	    3011,	    3015,	    3021,	    3023,	    3026,	    3032,	    3037,	    3042,	    3044,	    3058,	    3090,	    3229,	    3296,	    3308,	    3321,	    3338,	    3376,	    3432,	    3501,	    3533,	    3550,	    3593,	    3605,	    3607,	    3645,	    3653,	    3679,	    4545,	    4912,	    4915,	    4927,	    4943,	    4958,	    4999,	    5469,	    6108,	    6115,	    6133,	    6141,	    6153,	    6155,	    6165,	    6191,	    6197,	    6205,	    6213,	    6224,	    6251,	    6269,	    6282,	    6412,	    6422,	    6449,	    8039,	    8046,	    8103,	    8213,	    8249,	    1503,
        #電機機械
            1504,	1506,	1507,	1512,	1513,	1514,	1515,	1517,	1519,	1521,	1522,	1524,	1525,	1526,	1527,	1528,	1529,	1530,	1531,	1532,	1533,	1535,	1536,	1537,	1538,	1539,	1540,	1541,	1558,	1560,	1566,	1568,	1570,	1580,	1583,	1586,	1589,	1590,	1591,	1592,	1597,	1599,	2049,	2066,	2067,	2228,	2230,	2231,	2233,	2235,	2236,	2371,	3162,	3167,	3226,	3379,	3426,	4502,	4503,	4506,	4510,	4513,	4523,	4526,	4527,	4528,	4530,	4532,	4533,	4534,	4535,	4543,	4549,	4550,	4551,	4552,	4555,	4557,	4560,	4562,	5288,	6122,	6603,	6605,	6609,	8027,	8083,	8107,	8222,	8255,	8374,	8996,	9951,
        #電器電纜
            1603,	1604,	1605,	1608,	1609,	1611,	1612,	1614,	1615,	1616,	1617,	1618,	1626,	2061,	4609,	4930,
        #電腦及週邊
            1569,	2301,	2305,	2324,	2331,	2352,	2353,	2356,	2357,	2362,	2364,	2365,	2376,	2377,	2380,	2382,	2387,	2395,	2397,	2399,	2405,	2417,	2424,	2425,	2442,	2465,	3002,	3005,	3013,	3017,	3022,	3046,	3057,	3060,	3071,	3088,	3211,	3213,	3231,	3272,	3287,	3323,	3325,	3416,	3479,	3483,	3494,	3515,	3521,	3540,	3577,	3594,	3611,	3625,	3652,	3693,	3701,	3706,	4916,	4924,	4938,	4987,	5215,	5258,	5264,	5289,	5304,	5356,	5386,	5426,	5438,	5450,	5465,	5474,	5490,	6105,	6117,	6121,	6123,	6128,	6150,	6160,	6161,	6166,	6172,	6188,	6206,	6222,	6228,	6230,	6235,	6276,	6277,	6298,	6414,	6577,	6579,	8050,	8076,	8114,	8163,	8210,	8234,	8299,	8410,	9912,
        #半導體
            1437,	2302,	2303,	2311,	2325,	2329,	2330,	2337,	2338,	2342,	2344,	2351,	2363,	2369,	2379,	2388,	2401,	2408,	2434,	2436,	2441,	2449,	2451,	2454,	2458,	2481,	3006,	3014,	3016,	3034,	3035,	3041,	3054,	3073,	3094,	3105,	3122,	3141,	3169,	3188,	3189,	3227,	3228,	3257,	3259,	3260,	3264,	3265,	3268,	3317,	3372,	3374,	3413,	3438,	3443,	3519,	3527,	3529,	3532,	3536,	3545,	3553,	3555,	3556,	3567,	3579,	3581,	3583,	3588,	3661,	3675,	3680,	3686,	3707,	4919,	4947,	4952,	4966,	4968,	4971,	4973,	4991,	5269,	5272,	5274,	5285,	5302,	5305,	5314,	5344,	5347,	5351,	5425,	5455,	5468,	5471,	5483,	5487,	6103,	6104,	6129,	6138,	6147,	6182,	6198,	6202,	6223,	6229,	6233,	6237,	6239,	6243,	6257,	6261,	6271,	6287,	6291,	6411,	6415,	6435,	6451,	6457,	6462,	6485,	6488,	6494,	6510,	6525,	6531,	6532,	6533,	6552,	6568,	6594,	8016,	8024,	8040,	8054,	8081,	8086,	8088,	8110,	8131,	8150,	8261,	8271,	8277,
        #其他電子業
            1785,	2312,	2317,	2354,	2359,	2360,	2390,	2404,	2423,	2433,	2461,	2464,	2474,	2477,	2482,	2488,	2495,	2497,	3018,	3030,	3043,	3064,	3067,	3093,	3131,	3285,	3289,	3305,	3324,	3373,	3402,	3450,	3498,	3508,	3518,	3541,	3551,	3552,	3563,	3580,	3587,	3617,	3628,	3642,	3663,	3665,	4554,	5225,	5383,	5452,	5489,	5493,	5536,	6139,	6144,	6146,	6151,	6187,	6196,	6201,	6215,	6238,	6247,	6275,	6409,	6438,	6512,	8021,	8047,	8085,	8092,	8183,	8201,	8383,	8431,	8455,
        #光電業
            2323,	2340,	2349,	2374,	2393,	2406,	2409,	2426,	2438,	2448,	2466,	2475,	2486,	2489,	2491,	2499,	3008,	3019,	3024,	3031,	3038,	3049,	3050,	3051,	3059,	3066,	3128,	3149,	3230,	3297,	3339,	3356,	3362,	3383,	3406,	3428,	3434,	3437,	3441,	3452,	3454,	3455,	3481,	3490,	3504,	3514,	3516,	3522,	3523,	3531,	3535,	3557,	3561,	3562,	3576,	3591,	3615,	3622,	3623,	3629,	3630,	3666,	3669,	3673,	3685,	3691,	3698,	4729,	4933,	4934,	4935,	4942,	4944,	4956,	4960,	4972,	4976,	4995,	5230,	5234,	5243,	5245,	5251,	5259,	5281,	5315,	5371,	5392,	5432,	5443,	5484,	6116,	6120,	6125,	6131,	6164,	6167,	6168,	6176,	6209,	6225,	6226,	6234,	6244,	6246,	6278,	6289,	6405,	6419,	6431,	6443,	6456,	6477,	6548,	6560,	7402,	8049,	8064,	8069,	8087,	8105,	8111,	8215,	8240,
        #通訊網路
            2314,	2321,	2332,	2345,	2412,	2419,	2439,	2444,	2450,	2455,	2485,	2496,	2498,	3025,	3027,	3045,	3047,	3062,	3068,	3081,	3095,	3152,	3163,	3221,	3234,	3290,	3299,	3306,	3311,	3363,	3380,	3419,	3431,	3466,	3491,	3499,	3558,	3564,	3596,	3632,	3664,	3672,	3682,	3684,	3694,	3704,	4903,	4904,	4905,	4906,	4908,	4909,	4977,	4979,	4984,	5348,	5353,	5388,	6109,	6136,	6142,	6143,	6152,	6163,	6170,	6190,	6216,	6218,	6241,	6245,	6263,	6283,	6285,	6417,	6426,	6442,	6465,	6470,	6486,	6514,	8011,	8034,	8048,	8059,	8097,	8101,	8171,	8176,
        #資訊服務
            2427,	2453,	2468,	2471,	3029,	3083,	3085,	3130,	3293,	3546,	3570,	3687,	4946,	4953,	4965,	4994,	5201,	5202,	5203,	5205,	5209,	5210,	5211,	5212,	5278,	5287,	5310,	5403,	5410,	5478,	6111,	6112,	6140,	6148,	6169,	6180,	6183,	6214,	6221,	6231,	6240,	6404,	8044,	8099,	8416,	8472,	8477,
        #電子通路
            2347,	2414,	2430,	2459,	3010,	3028,	3033,	3036,	3048,	3055,	3209,	3224,	3232,	3312,	3315,	3360,	3444,	3528,	3702,	5434,	6107,	6113,	6118,	6145,	6154,	6189,	6227,	6265,	6270,	6281,	8032,	8067,	8068,	8070,	8084,	8096,	8112
    ]    
    
   
    table = raw[0:1] #先將table指派為原始raw之標頭
    
    #將table與符合elec_companies的公司代號的dataframe合併，組成新的table
    for i in range(len(elec_companies)):
        table = pd.concat([table, raw[raw[0] == str(elec_companies[i])]]) 
    table = table.reset_index(drop=True) #重設index
    
    return table

In [4]:
def compare_year(year1, year2):
    
    #取出表格的公司代號欄，轉換成set
    code1 = set(year1[0])
    code2 = set(year2[0])
       
    #取兩年度公司代號之交集
    code = code1.intersection(code2)
    code = list(code)
    
    #year1
    table1 = year1[year1[0].isin(code)]
    table1 = table1.reset_index(drop=True) #重設index  
    

    #year2
    table2 = year2[year2[0].isin(code)]
    table2 = table2.reset_index(drop=True) #重設index  
    
    
    return(table1, table2)

In [5]:
def compare_season(raw, s):
    if s == 3:
        s1 = raw[0]
        s2 = raw[1]
        s3 = raw[2]
        
        #取出表格的公司代號欄，轉換成set
        code1 = set(s1['公司代號'])
        code2 = set(s2['公司代號'])
        code3 = set(s3['公司代號'])

        #取三季度公司代號之交集
        code = code1.intersection(code2, code3)
        code = list(code)   

        #s1
        table1 = s1[s1['公司代號'].isin(code)]
#         table1 = s1[0:0] #先將table指派為原始raw之標頭
#         for i in range(len(code)):
#             table1 = pd.concat([table1, s1[s1['公司代號'] == str(code[i])]]) #將table與符合相同公司代號的dataframe合併，組成新的table
        table1 = table1.reset_index(drop=True) #重設index      

        #s2
        table2 = s2[s2['公司代號'].isin(code)]
#         table2 = s2[0:0] #先將table指派為原始raw之標頭
#         for i in range(len(code)):
#             table2 = pd.concat([table2, s2[s2['公司代號'] == str(code[i])]]) #將table與符合相同公司代號的dataframe合併，組成新的table
        table2 = table2.reset_index(drop=True) #重設index  

        #s3
        table3 = s3[s3['公司代號'].isin(code)]
#         table3 = s3[0:0] #先將table指派為原始raw之標頭
#         for i in range(len(code)):
#             table3 = pd.concat([table3, s3[s3['公司代號'] == str(code[i])]]) #將table與符合相同公司代號的dataframe合併，組成新的table
        table3 = table3.reset_index(drop=True) #重設index 

        return(table1, table2, table3)
    
#     elif s == 2:
#         s1 = raw[0]
#         s2 = raw[1]
        
#         #取出表格的公司代號欄，轉換成set
#         code1 = set(s1['公司代號'])
#         code2 = set(s2['公司代號'])

#         #取三季度公司代號之交集
#         code = code1.intersection(code2)
#         code = list(code)   

#         #s1
#         table1 = s1[0:0] #先將table指派為原始raw之標頭
#         for i in range(len(code)):
#             table1 = pd.concat([table1, s1[s1['公司代號'] == str(code[i])]]) #將table與符合相同公司代號的dataframe合併，組成新的table
#         table1 = table1.reset_index(drop=True) #重設index      

#         #s2
#         table2 = s2[0:0] #先將table指派為原始raw之標頭
#         for i in range(len(code)):
#             table2 = pd.concat([table2, s2[s2['公司代號'] == str(code[i])]]) #將table與符合相同公司代號的dataframe合併，組成新的table
#         table2 = table2.reset_index(drop=True) #重設index  

#         return(table1, table2)        

In [6]:
def category(raw):
    
    series = raw[0:1]
    category1 = [] #儲存公司代號、公司名稱、營業收入、營業毛利、基本每股盈餘之index
    category2 = [] #儲存所有類別的index
    
    for i in series:
        category2.append(series[i].name)
        if series[i][0] == '公司代號' or series[i][0] == '公司名稱' or series[i][0] == '營業收入' or series[i][0] == '營業毛利（毛損）淨額' or series[i][0] == '基本每股盈餘（元）':
            category1.append(i)    
    
    category1 = set(category1) #轉成set
    category2 = set(category2) #轉成set
    category3 = list(category2 - category1) #category3為不要的類別
    
    table = raw.drop(category3, axis=1)#刪除不要的類別  
    table.columns = [0, 1, 2, 3, 4]#重設column index    
    
    return table

In [7]:
def season(year):
    
    payload['year'] = year[0]    
    payload['season'] = year[1]  
    
    req = requests.post(url, data = payload, headers = head)
    req.encoding = 'utf8'
    
    table = pd.read_html(req.text)[3] #第四張表格才是我們要的資料   
    table = category(table)#篩選出eps, 營收, 毛利
    table = elec(table)#刪除非電子零件類股    
        
    return table

In [8]:
def growth(year):#y1新, y2舊

    #從year取出兩個年度
    year1 = year[0]
    year2 = year[1]
    
    #使y1, y2 公司名單相同
    y = compare_year(year1, year2)
    y1 = y[0]
    y2 = y[1]

    code = []#公司代號
    company = []#公司名稱
    income = []#營業收入
    profit = []#營業毛利（毛損）淨額
    eps = []#基本每股盈餘（元）

    for j in range(1, y1.shape[0]):
        code.append(y1[0][j])
        company.append(y1[1][j])
        income.append((float(y1[2][j]) - float(y2[2][j])) / float(y2[2][j]))#income成長率
        profit.append((float(y1[3][j]) - float(y2[3][j])) / float(y2[3][j]))#profit成長率

        #萬一eps為零，為了不讓分母為零，將其設為0.00000000000000000001
        if float(y2[4][j]) == 0:
            y2[4][j] = 0.00000000000000000001
        
        #分母必須為正，否則成長率有時會正負相反
        mother = float(y2[4][j])
        if mother < 0:
            mother = mother*(-1)
        eps.append((float(y1[4][j]) - float(y2[4][j])) / mother)#eps成長率

    s = {
        '公司代號':code,
        '公司名稱':company,
        '營業收入':income,
        '營業毛利（毛損）淨額':profit,
        '基本每股盈餘（元）':eps
    }
    s = pd.DataFrame(data = s)
        
    return s

#     return (year1, year2, y1, y2)

In [9]:
def multicore_season(y, pool):

    res = pool.map(season, y)#迭代相對應的次數
                                                        #partial目的：因pool只能傳入一個argument，故使用partial傳入multiple arguments
                                                        #http://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
    
    return res

In [10]:
def multicore_growth(s, y_new, y_old, pool):
    
    y_new = multicore_season(y_new, pool)
    y_old = multicore_season(y_old, pool)    
    
    y = []
    for i in range(s):
        y.append([y_new[i], y_old[i]])

    res = pool.map(growth, y)#迭代相對應的次數
                                                        #partial目的：因pool只能傳入一個argument，故使用partial傳入multiple arguments
                                                        #http://python.omics.wiki/multiprocessing_map/multiprocessing_partial_function_multiple_arguments
    
    return res

In [11]:
def eps(t, s):
    #3季
    if s == 3:
        t1 = t[0]
        t2 = t[1]
        t3 = t[2]
        
        #只留下eps欄
        t1 = t1.drop(['營業收入', '營業毛利（毛損）淨額'], axis=1)
        t2 = t2.drop(['營業收入', '營業毛利（毛損）淨額'], axis=1)
        t3 = t3.drop(['營業收入', '營業毛利（毛損）淨額'], axis=1)
        
        #重設column index
        t1.columns = ['公司代號', '公司名稱', '基本每股盈餘（元）']
        t2.columns = ['公司代號', '公司名稱', '基本每股盈餘（元）']
        t3.columns = ['公司代號', '公司名稱', '基本每股盈餘（元）']

        #使三季公司名單相同
        t = [t1, t2, t3]
        s = compare_season(t, s)
        s1 = s[0]
        s2 = s[1]
        s3 = s[2]

        for i in range(s1.shape[0]):
            if s1['基本每股盈餘（元）'][i] <= 0 or s2['基本每股盈餘（元）'][i] <= 0 or s3['基本每股盈餘（元）'][i] <= 0: 
                s1 = s1.drop([i], axis = 0)
                s2 = s2.drop([i], axis = 0)
                s3 = s3.drop([i], axis = 0)

        #排序
        s1 = s1.sort_values(by = '基本每股盈餘（元）', ascending = False)
        s2 = s2.sort_values(by = '基本每股盈餘（元）', ascending = False)
        s3 = s3.sort_values(by = '基本每股盈餘（元）', ascending = False)
        
        #重設index
        s1 = s1.reset_index(drop=True) 
        s2 = s2.reset_index(drop=True) 
        s3 = s3.reset_index(drop=True)         

        return (s1, s2, s3) 
    
#     #2季
#     elif s == 2:
#         t1 = t[0]
#         t2 = t[1]
        
#         #只留下eps欄
#         t1 = t1.drop(['營業收入', '營業毛利（毛損）淨額'], axis=1)
#         t2 = t2.drop(['營業收入', '營業毛利（毛損）淨額'], axis=1)
        
#         #重設column index
#         t1.columns = ['公司代號', '公司名稱', '基本每股盈餘（元）']
#         t2.columns = ['公司代號', '公司名稱', '基本每股盈餘（元）']

#         #使三季公司名單相同
#         t = [t1, t2]
#         s = compare_season(t, s)
#         s1 = s[0]
#         s2 = s[1]

#         for i in range(1, s1.shape[0]):
#             if s1['基本每股盈餘（元）'][i] < 0 or s2['基本每股盈餘（元）'][i] < 0: 
#                 s1 = s1.drop([i], axis = 0)
#                 s2 = s2.drop([i], axis = 0)

#         #重設index
#         s1 = s1.reset_index(drop=True) 
#         s2 = s2.reset_index(drop=True) 

#         #排序
#         s1 = s1.sort_values(by = '基本每股盈餘（元）', ascending = False)
#         s2 = s2.sort_values(by = '基本每股盈餘（元）', ascending = False)

#         return (s1, s2) 

In [12]:
def income(t, s):
    
    #3季
    if s == 3:
        t1 = t[0]
        t2 = t[1]
        t3 = t[2]
        
        #只留下eps欄
        t1 = t1.drop(['基本每股盈餘（元）', '營業毛利（毛損）淨額'], axis=1)
        t2 = t2.drop(['基本每股盈餘（元）', '營業毛利（毛損）淨額'], axis=1)
        t3 = t3.drop(['基本每股盈餘（元）', '營業毛利（毛損）淨額'], axis=1)
        
        #重設column index
        t1.columns = ['公司代號', '公司名稱', '營業收入']
        t2.columns = ['公司代號', '公司名稱', '營業收入']
        t3.columns = ['公司代號', '公司名稱', '營業收入']

        #使三季公司名單相同
        t = [t1, t2, t3]
        s = compare_season(t, s)
        s1 = s[0]
        s2 = s[1]
        s3 = s[2]

        for i in range(s1.shape[0]):
            if s1['營業收入'][i] <= 0 or s2['營業收入'][i] <= 0 or s3['營業收入'][i] <= 0: 
                s1 = s1.drop([i], axis = 0)
                s2 = s2.drop([i], axis = 0)
                s3 = s3.drop([i], axis = 0)

        #排序
        s1 = s1.sort_values(by = '營業收入', ascending = False)
        s2 = s2.sort_values(by = '營業收入', ascending = False)
        s3 = s3.sort_values(by = '營業收入', ascending = False)
        
        #重設index
        s1 = s1.reset_index(drop=True) 
        s2 = s2.reset_index(drop=True) 
        s3 = s3.reset_index(drop=True)         

        return (s1, s2, s3) 
    
#     #2季
#     elif s == 2:
#         t1 = t[0]
#         t2 = t[1]
        
#         #只留下eps欄
#         t1 = t1.drop(['基本每股盈餘（元）', '營業毛利（毛損）淨額'], axis=1)
#         t2 = t2.drop(['基本每股盈餘（元）', '營業毛利（毛損）淨額'], axis=1)
        
#         #重設column index
#         t1.columns = ['公司代號', '公司名稱', '營業收入']
#         t2.columns = ['公司代號', '公司名稱', '營業收入']

#         #使三季公司名單相同
#         t = [t1, t2]
#         s = compare_season(t, s)
#         s1 = s[0]
#         s2 = s[1]

#         for i in range(s1.shape[0]):
#             if s1['營業收入'][i] < 0 or s2['營業收入'][i] < 0: 
#                 s1 = s1.drop([i], axis = 0)
#                 s2 = s2.drop([i], axis = 0)

#         #重設index
#         s1 = s1.reset_index(drop=True) 
#         s2 = s2.reset_index(drop=True) 

#         #排序
#         s1 = s1.sort_values(by = '營業收入', ascending = False)
#         s2 = s2.sort_values(by = '營業收入', ascending = False)

#         return (s1, s2)     

In [13]:
def profit(t, s):
    #3季
    if s == 3:
        t1 = t[0]
        t2 = t[1]
        t3 = t[2]
        
        #只留下eps欄
        t1 = t1.drop(['基本每股盈餘（元）', '營業收入'], axis=1)
        t2 = t2.drop(['基本每股盈餘（元）', '營業收入'], axis=1)
        t3 = t3.drop(['基本每股盈餘（元）', '營業收入'], axis=1)
        
        #重設column index
        t1.columns = ['公司代號', '公司名稱', '營業毛利（毛損）淨額']
        t2.columns = ['公司代號', '公司名稱', '營業毛利（毛損）淨額']
        t3.columns = ['公司代號', '公司名稱', '營業毛利（毛損）淨額']

        #使三季公司名單相同
        t = [t1, t2, t3]
        s = compare_season(t, s)
        s1 = s[0]
        s2 = s[1]
        s3 = s[2]

        for i in range(1, s1.shape[0]):
            if s1['營業毛利（毛損）淨額'][i] <= 0 or s2['營業毛利（毛損）淨額'][i] <= 0 or s3['營業毛利（毛損）淨額'][i] <= 0: 
                s1 = s1.drop([i], axis = 0)
                s2 = s2.drop([i], axis = 0)
                s3 = s3.drop([i], axis = 0)

        #排序
        s1 = s1.sort_values(by = '營業毛利（毛損）淨額', ascending = False)
        s2 = s2.sort_values(by = '營業毛利（毛損）淨額', ascending = False)
        s3 = s3.sort_values(by = '營業毛利（毛損）淨額', ascending = False)
        
        #重設index
        s1 = s1.reset_index(drop=True) 
        s2 = s2.reset_index(drop=True) 
        s3 = s3.reset_index(drop=True)         

        return (s1, s2, s3) 
    
#     #2季
#     elif s == 2:
#         t1 = t[0]
#         t2 = t[1]
        
#         #只留下eps欄
#         t1 = t1.drop(['基本每股盈餘（元）', '營業收入'], axis=1)
#         t2 = t2.drop(['基本每股盈餘（元）', '營業收入'], axis=1)
        
#         #重設column index
#         t1.columns = ['公司代號', '公司名稱', '營業毛利（毛損）淨額']
#         t2.columns = ['公司代號', '公司名稱', '營業毛利（毛損）淨額']

#         #使三季公司名單相同
#         t = [t1, t2]
#         s = compare_season(t, s)
#         s1 = s[0]
#         s2 = s[1]

#         for i in range(1, s1.shape[0]):
#             if s1['營業毛利（毛損）淨額'][i] < 0 or s2['營業毛利（毛損）淨額'][i] < 0: 
#                 s1 = s1.drop([i], axis = 0)
#                 s2 = s2.drop([i], axis = 0)

#         #重設index
#         s1 = s1.reset_index(drop=True) 
#         s2 = s2.reset_index(drop=True) 

#         #排序
#         s1 = s1.sort_values(by = '營業毛利（毛損）淨額', ascending = False)
#         s2 = s2.sort_values(by = '營業毛利（毛損）淨額', ascending = False)

#         return (s1, s2) 

In [14]:
def intersection(e_raw, i_raw, p_raw):
    
    e1_raw = e_raw[0]
    i1_raw = i_raw[0]
    p1_raw = p_raw[0]
    
    e2_raw = e_raw[1]
    i2_raw = i_raw[1]
    p2_raw = p_raw[1]
    
    e3_raw = e_raw[2]
    i3_raw = i_raw[2]
    p3_raw = p_raw[2]
    
    #eps, income, profit皆有上榜才能留下，取其公司代號之交集
    e_code = set(e1_raw['公司代號'])
    i_code = set(i1_raw['公司代號'])
    p_code = set(p1_raw['公司代號'])
    code = e_code.intersection(i_code, p_code) #公司代號之交集
    code = list(code)  
    
    
    #重篩表格，把符合code的留下
    #eps
    e1_new = e1_raw[e1_raw['公司代號'].isin(code)]
    e1_new = e1_new.reset_index(drop=True) #重設index 
    
    e2_new = e2_raw[e2_raw['公司代號'].isin(code)]
    e2_new = e2_new.reset_index(drop=True) #重設index 
    
    e3_new = e3_raw[e3_raw['公司代號'].isin(code)]
    e3_new = e3_new.reset_index(drop=True) #重設index
    
    e_new = [e1_new, e2_new, e3_new]
    
    #income
    i1_new = i1_raw[i1_raw['公司代號'].isin(code)]
    i1_new = i1_new.reset_index(drop=True) #重設index 
    
    i2_new = i2_raw[i2_raw['公司代號'].isin(code)]
    i2_new = i2_new.reset_index(drop=True) #重設index 
    
    i3_new = i3_raw[i3_raw['公司代號'].isin(code)]
    i3_new = i3_new.reset_index(drop=True) #重設index
    
    i_new = [i1_new, i2_new, i3_new]
    
    #profit
    p1_new = p1_raw[p1_raw['公司代號'].isin(code)]
    p1_new = p1_new.reset_index(drop=True) #重設index 
    
    p2_new = p2_raw[p2_raw['公司代號'].isin(code)]
    p2_new = p2_new.reset_index(drop=True) #重設index 
    
    p3_new = p3_raw[p3_raw['公司代號'].isin(code)]
    p3_new = p3_new.reset_index(drop=True) #重設index
    
    p_new = [p1_new, p2_new, p3_new]    

    return (e_new, i_new, p_new)

In [15]:
def excel(eps, income, profit, s):
    #eps
    writer = pd.ExcelWriter('Eps.xlsx')
    eps[0].to_excel(writer,'season1')
    eps[1].to_excel(writer,'season2')
    if s > 2:
        eps[2].to_excel(writer,'season3')
    writer.save()
    
    #income
    writer = pd.ExcelWriter('Income.xlsx')
    income[0].to_excel(writer,'season1')
    income[1].to_excel(writer,'season2')
    if s > 2:
        income[2].to_excel(writer,'season3')
    writer.save()
    
    #profit
    writer = pd.ExcelWriter('Profit.xlsx')
    profit[0].to_excel(writer,'season1')
    profit[1].to_excel(writer,'season2')
    if s > 2:
        profit[2].to_excel(writer,'season3')
    writer.save()

<br><br><br><br><br><br><br><br><br>

<br>
<font size="7" face="monospace">參數設置</font>

<font size="5" face="monospace">設置年度</font>

In [1]:
%%time
year_new = [105, 4, 106, 2]
year_old = [104, 4, 105, 2]
seasons = 3

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


<font size="5" face="monospace">單年度單季資料</font>

In [17]:
%%time
if __name__ == '__main__':
    pool = mp.Pool()
    
    Y = inputs(year_new)
    
    s = multicore_season(Y, pool)
    
    pool.close
    pool.join
    
    s1 = s[0]
    s2 = s[1]
    if seasons > 2:
        s3 = s[2]    

CPU times: user 22.4 ms, sys: 21.7 ms, total: 44.1 ms
Wall time: 14.6 s


<font size="5" face="monospace">YoY三季成長資料</font>

In [18]:
%%time
if __name__ == '__main__':
    pool = mp.Pool()
    
    
    Y_new = inputs(year_new)
    Y_old = inputs(year_old)
    
    g = multicore_growth(seasons, Y_new, Y_old, pool)
    
    pool.close
    pool.join    
    
    g1 = g[0]
    g2 = g[1]
    if seasons > 2:
        g3 = g[2]     
    

CPU times: user 56.7 ms, sys: 34.6 ms, total: 91.3 ms
Wall time: 29.6 s


<font size="5" face="monospace">eps or 毛利淨額 or 營收</font>

In [19]:
%%time
e = eps(g, seasons)
e1 = e[0]
e2 = e[1]
if seasons > 2:
    e3 = e[2]  

CPU times: user 641 ms, sys: 18.9 ms, total: 660 ms
Wall time: 656 ms


In [20]:
%%time
i = income(g, seasons)
i1 = i[0]
i2 = i[1]
if seasons > 2:
    i3 = i[2]  

CPU times: user 546 ms, sys: 9.32 ms, total: 555 ms
Wall time: 549 ms


In [21]:
%%time
p = profit(g, seasons)
p1 = p[0]
p2 = p[1]
if seasons > 2:
    p3 = p[2]  

CPU times: user 551 ms, sys: 9.05 ms, total: 560 ms
Wall time: 555 ms


<font size="5" face="monospace">交集</font>

In [22]:
%%time
intersec = intersection(e, i, p)

e_intersec = intersec[0]
e1_intersec = e_intersec[0]
e2_intersec = e_intersec[1]
e3_intersec = e_intersec[2]

i_intersec = intersec[1]
i1_intersec = i_intersec[0]
i2_intersec = i_intersec[1]
i3_intersec = i_intersec[2]

p_intersec = intersec[2]
p1_intersec = p_intersec[0]
p2_intersec = p_intersec[1]
p3_intersec = p_intersec[2]

CPU times: user 14.4 ms, sys: 2.56 ms, total: 16.9 ms
Wall time: 15.2 ms


<font size="5" face="monospace">輸出成excel</font>

In [34]:
%%time
excel(e, i, p, seasons)

CPU times: user 244 ms, sys: 39.6 ms, total: 283 ms
Wall time: 310 ms


In [35]:
%%time
excel(e_intersec, i_intersec, p_intersec, seasons)

CPU times: user 97.8 ms, sys: 15.3 ms, total: 113 ms
Wall time: 113 ms


<br>
<font size="7" face="monospace">演示區</font>

In [17]:
s1

,0,1,2,3,4
0,公司代號,公司名稱,營業收入,營業毛利（毛損）淨額,基本每股盈餘（元）
1,1471,首利,1187138,-58300,-1.91
2,1582,信錦,9138485,2172780,6.06
3,2059,川湖,4466110,2550860,16.49
4,2308,台達電,214355571,59493731,7.24
5,2313,華通,45515199,5775144,1.36
6,2316,楠梓電,4900284,474106,0.17
7,2327,國巨,29616351,7258354,6.83
8,2328,廣宇,18412296,1458032,2.15
9,2355,敬鵬,23939699,4383850,6.26


In [18]:
g1

,公司代號,公司名稱,基本每股盈餘（元）,營業收入,營業毛利（毛損）淨額
0,6449,鈺邦科技,1.045296e-02,0.138301,0.051194
1,3045,台灣大,-2.256944e-02,0.004333,0.041198
2,3501,維熹科技,-6.012658e-02,-0.092239,0.108303
3,5285,界霖,-1.066667e-01,-0.034263,0.025918
4,6108,競國實業,2.533333e+01,0.098413,0.556579
5,4526,東台,-7.830189e-01,0.063026,-0.132279
6,2434,統懋,-5.632184e-01,0.051733,-0.952674
7,1528,恩德,-3.333333e+00,0.178785,0.360882
8,6422,君耀-KY,1.428571e-01,0.166709,0.161437
9,2368,金像電,4.444444e-01,0.002878,0.057189


In [41]:
e3

,公司代號,公司名稱,基本每股盈餘（元）
86,2496,卓越成功,28.882353
94,8011,台通,20.500000
14,3645,達邁,19.000000
77,6120,達運,15.000000
81,6226,光鼎電子,8.333333
29,1612,宏泰,7.600000
27,1608,華榮,7.428571
82,6456,GIS-KY,7.042105
56,8110,華東,7.000000
26,8374,羅昇,5.850000


In [23]:
e3_intersec

,公司代號,公司名稱,基本每股盈餘（元）
0,2496,卓越成功,28.882353
1,8011,台通,20.500000
2,3645,達邁,19.000000
3,1612,宏泰,7.600000
4,8110,華東,7.000000
5,3013,晟銘電子,3.125000
6,2337,旺宏,2.044444
7,6141,柏承科技,2.011494
8,3419,譁裕實業,1.750000
9,2439,美律,1.346939


In [36]:
filter = s1[1] == '玉晶光' 
s1[filter] 

,0,1,2,3,4
357,3406,玉晶光,7030362,1746967,-1.54


In [39]:
filter = s1[1] == '玉晶光' 
s1[filter] 

,0,1,2,3,4
357,3406,玉晶光,9047416,729858,-11.30


In [34]:
filter = s2[1] == '玉晶光' 
s2[filter] 

,0,1,2,3,4
357,3406,玉晶光,1510337,525551,0.08


In [40]:
filter = s2[1] == '玉晶光' 
s2[filter] 

,0,1,2,3,4
353,3406,玉晶光,1155340,63006,-3.17


In [37]:
filter = s3[1] == '玉晶光' 
s3[filter] 

,0,1,2,3,4
357,3406,玉晶光,2926132,1133316,1.56


In [41]:
filter = s3[1] == '玉晶光' 
s3[filter] 

,0,1,2,3,4
357,3406,玉晶光,2790616,301806,-5.88


In [27]:
filter = g1['公司名稱'] == '玉晶光' 
g1[filter] 

,公司代號,公司名稱,基本每股盈餘（元）,營業收入,營業毛利（毛損）淨額
356,3406,玉晶光,0.863717,-0.222943,1.393571


In [28]:
filter = g2['公司名稱'] == '玉晶光' 
g2[filter] 

,公司代號,公司名稱,基本每股盈餘（元）,營業收入,營業毛利（毛損）淨額
352,3406,玉晶光,1.025237,0.307266,7.341285


In [29]:
filter = g3['公司名稱'] == '玉晶光' 
g3[filter] 

,公司代號,公司名稱,基本每股盈餘（元）,營業收入,營業毛利（毛損）淨額
356,3406,玉晶光,1.265306,0.048561,2.755114


In [33]:
filter = i2['公司名稱'] == '玉晶光' 
i2[filter] 

,公司代號,公司名稱,營業收入


<br>
<font size="7" face="monospace">測試區</font>

In [20]:
def parallel_runs():
    pool = mp.Pool(processes=4)
    input_list = [1, 2, 3, 4]
    prod_x=partial(prod_xy, y=10) #prod_x has only one argument x (y is fixed to 10)    
    result_list = pool.map(prod_x, input_list) 
    print(result_list)
   
def prod_xy(x,y):
    return x * y

if __name__ == '__main__':
    parallel_runs()

[10, 20, 30, 40]


In [28]:
import time
import threading as td    
import multiprocessing as mp

def job(q):
    res = 0
    for i in range(100000):
        res += i +i**2 + i**3
    q.put(res)
    
def multicore():
    q = mp.Queue()
    
    p1 = mp.Process(target = job, args = (q, ))
    p2 = mp.Process(target = job, args = (q, ))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()

    res1 = q.get()
    res2 = q.get()

    print('process:', res1 +res2)    

def normal():    
    res = 0
    for j in range(2):
        for i in range(100000):
            res += i +i**2 + i**3
    print('normal:', res)
    
def multithread():
    q = mp.Queue()
    
    t1 = td.Thread(target = job, args = (q, ))
    t2 = td.Thread(target = job, args = (q, ))
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()

    res1 = q.get()
    res2 = q.get()

    print('thread:', res1 +res2)  
    
    
if __name__ == '__main__':
    st = time.time()
    
    normal()
    st1 = time.time()
    print('normal time:', st1 -st)
    
    multithread()
    st2 = time.time()
    print('thread time:', st2 -st)
    
    multicore()
    print('process time:', time.time() -st2)   

normal: 49999666671666600000
normal time: 0.2309279441833496
thread: 49999666671666600000
thread time: 0.49998998641967773
process: 49999666671666600000
process time: 0.14261293411254883


In [52]:
import multiprocessing as mp

def job(x):
    return x*x

def multicore():
    pool = mp.Pool()
    res = pool.map(job, range(10))
    print(res)

    
if __name__ == '__main__':
    multicore()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


Process ForkPoolWorker-42:
Process ForkPoolWorker-43:
Process ForkPoolWorker-44:
Process ForkPoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/AndrewHowCool/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/AndrewHowCool/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/AndrewHowCool/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/AndrewHowCool/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/AndrewHowCool/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/Users/AndrewHowCool/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/AndrewHowCool/an

In [ ]:
%%time
payload['year'] = '106'
payload['season'] = '01'    
req = requests.post(url, data = payload, headers = head)%time
req.encoding = 'utf8'%time

table = pd.read_html(req.text)[3]%time
table

In [31]:
table

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,公司代號,公司名稱,營業收入,營業成本,營業毛利（毛損）,未實現銷貨（損）益,已實現銷貨（損）益,營業毛利（毛損）淨額,營業費用,其他收益及費損淨額,...,其他綜合損益（淨額）,合併前非屬共同控制股權綜合損益淨額,本期綜合損益總額,淨利（淨損）歸屬於母公司業主,淨利（淨損）歸屬於共同控制下前手權益,淨利（淨損）歸屬於非控制權益,綜合損益總額歸屬於母公司業主,綜合損益總額歸屬於共同控制下前手權益,綜合損益總額歸屬於非控制權益,基本每股盈餘（元）
1,1101,台泥,17724112,16138503,1585609,--,--,1585609,1101720,--,...,-828511,--,-733984,220995,--,-126468,-333525,--,-400459,0.06
2,1102,亞泥,13038960,12040729,998231,0,201,998432,592180,--,...,-8936360,--,-8741901,199130,--,-4671,-8505796,--,-236105,0.06
3,1103,嘉泥,541218,544548,-3330,--,--,-3330,87944,--,...,126972,--,-62013,-188620,--,-365,-66938,--,4925,-0.29
4,1104,環球水泥,1105803,990332,115471,--,--,115471,83813,--,...,20078,--,363061,345289,--,-2306,365367,--,-2306,0.55
5,1108,幸福水泥,995636,804827,190809,--,--,190809,57790,--,...,-4655,--,99716,103392,--,979,98737,--,979,0.26
6,1109,信大水泥,875999,807187,68812,--,--,68812,95228,--,...,-923,--,-34127,-11582,--,-21622,-9154,--,-24973,-0.03
7,1110,東泥,383670,347147,36523,--,--,36523,20739,--,...,60334,--,276947,117463,--,99150,177798,--,99149,0.21
8,1201,味全公司,3518095,2674670,843425,--,--,843425,1374779,--,...,-57598,--,-140059,-108469,--,26008,-165965,--,25906,-0.21
9,1203,味王公司,1547878,1191232,356646,0,0,356646,210818,--,...,-14359,--,108602,88980,--,33981,71736,--,36866,0.37


In [1]:
key = {}
for i in range(3):
    key['test' + str(i)] = i

In [3]:
key

{'test0': 0, 'test1': 1, 'test2': 2}

In [18]:
a, b, c = 1, 1, 1

In [19]:
b

1

In [20]:
c

1